# PG-REPACK

## Description

This is a very commonly used optimization for `FULL VACUUM` (or `CLUSTER`) process.

It performs an almost identical flow but without holding the `AccessExclusiveLock` for so long.

## Flow

The flow is very similar to `FULL VACUUM` except instead of locking the table through the process it's creating triggers on the original table and due to the end of the process it applies those new changes and swaps the tables.

**In Detail:**
1. Create a log table to record changes made to the original table
1. Add a trigger onto the original table, logging INSERTs, UPDATEs and DELETEs into our log table
1. Create a new table containing all the rows in the old table
1. Build indexes on this new table
1. Apply all changes which have accrued in the log table to the new table
1. Swap the tables, including indexes and toast tables, using the system catalogs
1. Drop the original table


`pg_repack` will only hold an `AccessExclusiveLock` for a short period during initial setup (steps 1 and 2 above) and during the final swap-and-drop phase (steps 6 and 7). For the rest of its time, pg_repack only needs to hold an `AccessShareLock` on the original table, meaning INSERTs, UPDATEs, and DELETEs may proceed as usual.

## Restrictions

- Requires the table to have a primary key or some not-null unique constraints to be able to reproduce the changes from the log table.
- Doesn't work on temporary tables.
- Doesn't work on tables that have global indexes.

Most of the time it's not an issue because usually production tables have primary keys and you most probably don't want to use `FULL VACUUM` on temp tables.

## Install

```BASH
apt install pg-repack
psql -c "CREATE EXTENSION pg_repack"
```

## Use

```SQL
DROP TABLE IF EXISTS tbl CASCADE;
CREATE TABLE tbl (
    id INTEGER PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    data INT
);

INSERT INTO tbl(data)
SELECT 0 AS data
FROM generate_series(1,1000000);

SELECT pg_size_pretty(pg_relation_size('tbl'));
DELETE FROM tbl WHERE id BETWEEN 2 AND 900000;
```

```BASH
pg_repack -t tbl
psql -c "SELECT pg_size_pretty(pg_relation_size('tbl'));"
```